# 3 Neural Network Basics
**Objective:** Explore the basics of neural networks and how to build them using PyTorch.

In [4]:
from dataclasses import dataclass

from chinese_checkers.simulation.GameSimulationCatalog import GameSimulationCatalog
from chinese_checkers.game.ChineseCheckersGame import ChineseCheckersGame
from torch import tensor, stack, zeros_like

In [5]:
# encoding function from 02_Data_Preprocessing.ipynb
def generate_tensor_for_game(game: ChineseCheckersGame) -> tensor:
    all_positions = game.board.hexagram_points
    encoded_state = tensor([
        [
            1 if position in player.positions else 0
            for position in all_positions
        ]
        for player in game.players
    ])

    return encoded_state

### 2.1 Neural Network Approach for Chinese Checkers

#### Objective:
Train a model to assess the probability of a win for each player based on a given game state in Chinese Checkers.

#### **Structure**:

1. **Input Layer**:
    - Represents the game state. Depending on the representation, this could be a one-dimensional array (flattened) or a two-dimensional grid resembling the board.

2. **Output Layer**:
    - Comprises two neurons, one for each player. Each produces a score indicating the likelihood of that player winning from the given game state.
    - Scores are continuous values between 0 (certain loss) and 1 (certain win).

3. **Scoring Mechanism**:
    - For a given game state, if Player 1's neuron outputs a score of `x`, then Player 2's score is `1 - x`. This ensures the scores are complementary and the sum is always 1.
    - Early in the game, scores are expected to be close to 0.5 for both players, indicating no clear advantage.

#### **Training**:

1. **Data Collection**:
    - Amass a large dataset of game states with their outcomes, either through simulation or from historical data.

2. **Labeling**:
    - Assign scores to each game state for each player. This score denotes the probability of each player winning from that state.

3. **Optimization**:
    - Employ an appropriate loss function, such as mean squared error, to train the neural network.
    - The training goal is to minimize the difference between the predicted scores and the true outcomes of the game states.

#### **Interpretation**:

- A score closer to 1 for Player 1 denotes a high likelihood of Player 1 winning from that state. Conversely, a score near 0 signifies a higher chance for Player 2's victory.


In [6]:
@dataclass()
class GameLabel:
    player_1_wins: int = 0
    player_2_wins: int = 0

    def update_player_wins(self, player_index):
        if player_index == 0:
            self.player_1_wins += 1
        elif player_index == 3:
            self.player_2_wins += 1

In [9]:
import logging
import time

# Setup logging configuration
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

catalog = GameSimulationCatalog("D:\chinese_checkers_games")
metadata_list = catalog.list_available_metadata()

game_labels = {}

for metadata in metadata_list:
    if metadata.version == "1.0":
        game_winner = metadata.winning_player
        game_winner_key = "player_1_wins" if game_winner == 0 else "player_2_wins"

        # Start the timer
        start_time = time.time()

        simulation_count = 0

        for simulation in catalog.load_simulations_by_metadata(metadata):
            for game in set(simulation.to_game_sequence()):
                if game not in game_labels:
                    game_labels[game] = GameLabel()
                game_labels[game].update_player_wins(game_winner)

            simulation_count += 1
            if simulation_count % 1000 == 0:
                logging.info(f"Processed {simulation_count} simulations.")
            if simulation_count == 10000:
                break

        # Calculate the time taken to process one metadata
        end_time = time.time()
        elapsed_time = end_time - start_time
        logging.info(f"Time taken to process one metadata: {elapsed_time:.4f} seconds")

2023-10-29 22:27:06,070 - Initialized GameSimulationCatalog at D:\chinese_checkers_games
2023-10-29 22:27:21,164 - Processed 1000 simulations.
2023-10-29 22:27:35,943 - Processed 2000 simulations.
2023-10-29 22:27:50,341 - Processed 3000 simulations.
2023-10-29 22:28:03,845 - Processed 4000 simulations.
2023-10-29 22:28:17,688 - Processed 5000 simulations.
2023-10-29 22:28:31,873 - Processed 6000 simulations.
2023-10-29 22:28:46,323 - Processed 7000 simulations.
2023-10-29 22:29:01,341 - Processed 8000 simulations.
2023-10-29 22:29:13,762 - Processed 9000 simulations.
2023-10-29 22:29:29,500 - Processed 10000 simulations.
2023-10-29 22:29:29,534 - Time taken to process one metadata: 141.2942 seconds
2023-10-29 22:29:41,912 - Processed 1000 simulations.
2023-10-29 22:29:58,069 - Processed 2000 simulations.
2023-10-29 22:30:10,258 - Processed 3000 simulations.
2023-10-29 22:30:27,405 - Processed 4000 simulations.
2023-10-29 22:30:39,958 - Processed 5000 simulations.
2023-10-29 22:30:52,4

In [5]:
import pickle

with open("D:\game_labels.pkl", "wb") as file:
    pickle.dump(game_labels, file)

MemoryError: 

So the hashing approach is not working.  It takes wayyy too much memory. We will need to find a game hashing function that is more efficient and does not waste memory.  I will consider Zobrist Hashing and see what refactoring I can do to the Chinese Checkers package to support easier data access and effective hashing.